<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

In [32]:
import pandas as pd
""" buscarNomeEstadoPorUF """
def buscarNomeEstadoPorUF(uf):
    estados = pd.read_csv('dataset-support/estados.csv')
    try:
        estado = estados.query('uf == "{}"'.format(uf))
        return estado.iloc[0].nome
    except:
        return None
    
def buscarUfEstadoPorNome(nome):
    estados = pd.read_csv('dataset-support/estados.csv')
    try:
        estado = estados.query('nome == "{}"'.format(nome))
        return estado.iloc[0].uf
    except:
        return None
    
def listarNomesEstados():
    estados = pd.read_csv('dataset-support/estados.csv')
    return list(estados[["nome"]].values.flatten())


# Dados gerais

In [35]:
import jupyter_require
#%load_ext jupyter_require
#%requirejs d3 https://d3js.org/d3.v5.min
from jupyter_datatables import init_datatables_mode
import plotly.graph_objects as go
init_datatables_mode()
dadosGerais = pd.read_csv('datasets/all_total.csv')
dadosGerais.columns = ["Localidade", "Tempo", "Confirmados", "Mortes"]
# tratamento de nomes de localidades
display(dadosGerais)

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

,Localidade,Tempo,Confirmados,Mortes
0,Afghanistan,100,16509,270
1,Albania,86,1164,33
2,Algeria,99,9626,667
3,Andorra,93,844,51
4,Angola,75,86,4
5,Antigua and Barbuda,82,26,3
6,Argentina,92,18319,569
7,Armenia,94,10009,158
8,Australia,129,7229,102
9,Austria,99,16759,669


# Casos confirmados VS óbitos

Para determinada localidade: Comparativo entre números de casos DIÁRIOS confirmados e mortes ao longo do tempo.
- insight: o quanto aquela localidade consegue absorver casos confirmados, minimizando mortes.

In [8]:
import plotly
import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import widgets
import pandas as pd

def filtarDataSetPorUf(input):
    dataset = pd.read_csv('datasets/all_diario.csv')
    datasetFiltrado = dataset.query('Country == "{}"'.format(input))
    return datasetFiltrado

def setPlot(dataset, nome):
    fig = go.Figure()
    datasetAux = dataset[['Date', 'Country',"Confirmed","Deaths"]]
    fig.add_scatter(x=datasetAux.Date, y=datasetAux.Confirmed,name='Confirmed')
    fig.add_scatter(x=datasetAux.Date, y=datasetAux.Deaths, name='Deaths')
    fig.update_layout(title=nome, xaxis_title="Data da observação",yaxis_title="Número de casos no dia")
    fig.show()
    return fig
    
dropDown = widgets.Dropdown(
    options=listarNomesEstados(),
    description='Estado:',
    disabled=False,
)
output = widgets.Output()
display(dropDown, output)


def observeDropDown(change):
    with output:
        uf = buscarUfEstadoPorNome(change['new'])
        df = filtarDataSetPorUf(uf)
        nome = buscarNomeEstadoPorUF(uf)
        setPlot(df, nome)

dropDown.observe(observeDropDown, names='value')
## plot
nome = listarNomesEstados()[0]
uf = buscarUfEstadoPorNome(nome)
df = filtarDataSetPorUf(uf)

fig = setPlot(df, nome)


Dropdown(description='Estado:', options=('Rondônia', 'Acre', 'Amazonas', 'Roraima', 'Pará', 'Amapá', 'Tocantin…

Output()

# Comparativo de crescimento no número de casos confirmados entre localidades
- A unidade de tempo no eixo X representa uma "normalização" nas datas do levantamento do número de casos. Deste modo, a mesma unidade de tempo N para diferentes localidades podem representar um dia diferente. 

In [36]:
import plotly
import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import widgets
import pandas as pd
dataset = pd.read_csv('datasets/all_acumulado.csv')
fig2 = px.line(dataset
        , title='Crescimento do número de casos ao longo do tempo (escala linear)'
        
        , x="Time"
        , y="Confirmed"
        , color="Country"
        , line_group="Country"
        , hover_name="Country"
        , labels = {"Time":"Unidade de tempo","Confirmed":"Número de casos confirmados"})
#        ,log_y=True
#        ,log_x=True
fig2.show()


fig3 = px.line(dataset
        , title='Crescimento do número de casos ao longo do tempo (escala Log eixo Y)'
        
        , x="Time"
        , y="Confirmed"
        , color="Country"
        , line_group="Country"
        , hover_name="Country"
        ,log_y=True
#        ,log_x=True
        , labels = {"Time":"Unidade de tempo","Confirmed":"Número de casos confirmados"})

fig3.show()